# Konkatenacje, złączenia i grupowanie

Julian Zubek, 2015, [DELab UW](http://www.delab.uw.edu.pl/)

Analizowane dane często pochodzą z różnych źródeł, są zapisane w różnych formach i muszą zostać przekształcone do pożądanej przez nas postaci. Biblioteka pandas oferuje wiele podstawowych funkcji, ułatwiajacych takie przekształcenia. Przyjrzyjmy się podstawowym operacjom.

## Konkatenacja

In [3]:
import pandas as pd

d = pd.DataFrame({'a': [0,1,2], 'b': [3,4,5]}, index=['x1', 'x2', 'x3'])
d

,a,b
x1,0,3
x2,1,4
x3,2,5


In [4]:
d.transpose()

,x1,x2,x3
a,0,1,2
b,3,4,5


Do sklejania serii lub tabel razem wzdłuż pojedynczej osi służy funkcja concat.

In [5]:
a = pd.Series([1,2,3])
b = pd.Series([4,5,6])
c = pd.Series([7,8,9])

pd.concat((a,b,c))

0    1
1    2
2    3
0    4
1    5
2    6
0    7
1    8
2    9
dtype: int64

In [6]:
pd.concat((a,b,c), ignore_index=True)

0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
8    9
dtype: int64

In [7]:
pd.concat((a,b,c), axis=1)

,0,1,2
0,1,4,7
1,2,5,8
2,3,6,9


In [8]:
pd.concat((pd.concat((a,b,c), axis=1), a), axis=1)

,0,1,2,0
0,1,4,7,1
1,2,5,8,2
2,3,6,9,3


In [9]:
pd.concat((d, d.transpose()))

,a,b,x1,x2,x3
x1,0,3,NaN,NaN,NaN
x2,1,4,NaN,NaN,NaN
x3,2,5,NaN,NaN,NaN
a,NaN,NaN,0,1,2
b,NaN,NaN,3,4,5


In [10]:
dt = d.transpose()
d.columns = ["x1", "x2"]
pd.concat((d, dt))

,x1,x2,x3
x1,0,3,NaN
x2,1,4,NaN
x3,2,5,NaN
a,0,1,2
b,3,4,5


Przepis na skuteczną konkatencję:
1. Wybierz odpowiednie wiersze i kolumny z pierwotnych tabel (.loc, .iloc).
2. W razie potrzeby transponuj tabele wejściowe (.transpose).
3. Znormalizuj odpowiedno indeksy kolumn i wierszy (.index, .columns).
4. Użyj funkcji .concat podając odpowiednią koleność źródeł i wybierając odpowiednią oś (.concat((...), axis=...).

## Ćwiczenie

Wczytaj tabelę "../dane/telephone_calls.csv". Są to fikcyjne dane na temat połączeń telefonicznych pomiędzy różnymi osobami. Na jej podstawie stwórz tabelę zawierającą wyłącznie nazwiska i numery telefonów w dwóch kolumnach "Nazwisko" i "Nr telefonu". Tabela powinna zawierać wszystkie numery i nazwiska, które pojawiły się w pierwotnej tabeli.

Z niektórych numerów wykonano wiele połączeń -- w nowej tabeli znajdą się zduplikowane rekordy. Możemy łatwo usunąć je korzystając z funkcji: .drop_duplicates()

## Złączenia

Wczytamy dane dotyczące lotnisk na świecie.

<img src="openflights-apdb.png" />

In [2]:
import pandas as pd

airports = pd.read_csv("../dane/airports.csv", header=None)
airports.columns = ["Airport ID", "Name", "City", "Country", "IATA/FAA", "ICAO", "Latitude", "Longitude",
                    "Altitude", "Timezone", "DST", "Tz database time zone"]

Z innego źródła pozyskaliśmy dane na temat powierzchni poszczególnych państw.

In [3]:
areas = pd.read_csv("../dane/area.csv")
areas.head()

,Country Code,Country Name,Land area
0,AFG,Afghanistan,652230
1,ALB,Albania,27400
2,DZA,Algeria,2381740
3,ASM,American Samoa,200
4,AND,Andorra,470


In [5]:
s1 = areas["Land area"]
s1.index = areas["Country Name"]
s1 = pd.DataFrame(s1)
s1.head()

,Land area
Country Name,
Afghanistan,652230
Albania,27400
Algeria,2381740
American Samoa,200
Andorra,470


In [6]:
s2 = airports["Country"].value_counts()
s2.name = "Number of airports"
s2 = pd.DataFrame(s2)
s2.head()

,Number of airports
United States,1697
Canada,435
Germany,321
Australia,263
Russia,249


In [8]:
cla = pd.merge(s1, s2, right_index=True, left_index=True)
cla.head()

,Land area,Number of airports
United States,9147420,1697
Canada,9093510,435
Germany,348610,321
Australia,7682300,263
France,547660,233


Często spotykaną operacją na tabeli jest sortowanie według kolumny:

In [10]:
cla.sort("Number of airports").head()

,Land area,Number of airports
Tuvalu,30,1
Barbados,430,1
Benin,110620,1
Monaco,2,1
Isle of Man,570,1


In [11]:
cla.sort("Number of airports", ascending=False).head()

,Land area,Number of airports
United States,9147420,1697
Canada,9093510,435
Germany,348610,321
Australia,7682300,263
France,547660,233


## Ćwiczenie

Dodaj do tabeli cla nową kolumną z gęstością lotnisk na km$^2$. Wypisz 20 państw z największą gęstością lotnisk.

## Ćwiczenie

Wypisz 20 państw z największą gęstością lotnisk spośród państw o powierzchni większej niż 30000 km$^2$.

Tym razem wczytamy dane dotyczące tras lotniczych.

<img src="openflights-routedb.png" />

In [4]:
routes = pd.read_csv("../dane/routes.csv", header=None, na_values=["\\N"])
routes.columns = ["Airline",
                  "Airline ID",
                  "Source airport",
                  "Source airport ID",
                  "Destination airport",
                  "Destination airport ID",
                  "Codeshare",
                  "Stops",
                  "Equipment"]

Kolumna "Airport ID" z tabeli airports jest kompatybilna z kolumnami "Source airport ID" oraz "Destination airport ID" z tabeli routes. Przyjrzymy się ponownie funkcji .merge.

Zadanie: złącz dane z obu tabeli tak, żeby otrzymać pojedynczą tabelę zawierającą następujące kolumny:
* Airline
* Source airport
* Source country
* Destination airport
* Destination country

In [15]:
r = airports[["Airport ID", "Name", "Country"]]
r.columns = ["Airport ID", "Source airport", "Source country"]

l = routes[["Airline", "Source airport ID", "Destination airport ID"]]
l = pd.merge(l, r, left_on="Source airport ID", right_on="Airport ID")
l = l.drop('Airport ID', 1)

l.head()

,Airline,Source airport ID,Destination airport ID,Source airport,Source country
0,2B,2965,2990,Sochi,Russia
1,7J,2965,2979,Sochi,Russia
2,9U,2965,1735,Sochi,Russia
3,B2,2965,2954,Sochi,Russia
4,HY,2965,2983,Sochi,Russia


In [16]:
r.columns = ["Airport ID", "Destination airport", "Destination country"]

source_destination = pd.merge(l, r, left_on="Destination airport ID", right_on="Airport ID")
source_destination = source_destination.drop('Airport ID', 1)

source_destination.head()

,Airline,Source airport ID,Destination airport ID,Source airport,Source country,Destination airport,Destination country
0,2B,2965,2990,Sochi,Russia,Kazan,Russia
1,2B,2966,2990,Astrakhan,Russia,Kazan,Russia
2,2B,2968,2990,Balandino,Russia,Kazan,Russia
3,2B,4029,2990,Domododevo,Russia,Kazan,Russia
4,S7,4029,2990,Domododevo,Russia,Kazan,Russia


## Ćwiczenie

Zmodyfikuj powyższy kod tak, aby do tabeli source_destination dodać dodatkowe kolumny "Source altitude" i "Destination altitude" zawierające wysokości lotnisk.

## Ćwiczenie

Dodaj do tabeli source_destination nową kolumnę, zawierającą różnicę wysokości pomiędzy lotniskiem startowym a docelowym.

Wskazówka: funkcja abs() zwraca wartość bezwzględną liczby.

## Grupowanie

Tabele pandas pozwalają na organizowanie rekordów w grupy, tworząc hierarchiczną strukturę danych. Zademonstrujemy to na podstawie danych o lotniskach.

In [23]:
tz_airports = airports.groupby("Timezone")
tz_airports.groups[4.5] #Indeksy lotnisk w strefie czasowej GMT +4.5

[2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 4400,
 5579,
 5611,
 6074,
 6075,
 6076,
 6442,
 6700,
 6717,
 6718,
 6838,
 7341,
 7393]

Po pogrupowaniu obiektów można policzyć określone statystyki wewnątrz każdej z grup. Przykładowo, przyjrzyjmy się średniej długości geograficznej lotnisk położonych w tej samej strefie czasowej.

In [30]:
tz_airports["Longitude"].mean()

Timezone
-12.00     -157.349778
-11.00     -171.422077
-10.00     -152.356720
-9.50      -139.717511
-9.00      -152.126476
-8.00      -121.610808
-7.00      -110.999733
-6.00       -93.100722
-5.00       -78.078404
-4.50       -67.071296
-4.00       -62.770530
-3.50       -55.881816
-3.00       -53.055255
-2.00       -36.407274
-1.00       -25.801148
 0.00        -5.603453
 1.00         9.144554
 2.00        28.161467
 3.00        40.005081
 3.50        52.726326
 4.00        45.967628
 4.50        67.004506
 5.00        67.015282
 5.50        80.088905
 5.75        84.503532
 6.00        73.387658
 6.50        96.361407
 7.00       101.573618
 8.00       113.613455
 9.00       130.741661
 9.50       135.734217
 10.00      144.444115
 10.50      159.077000
 11.00      158.714859
 11.50      167.938742
 12.00      134.109036
 12.75     -176.338889
 13.00     -173.471427
Name: Longitude, dtype: float64

## Ćwiczenie

Oblicz minimalną, średnią i maksymalną wysokość lotnisk w poszczególnych krajach.

## Tabele krzyżowe

Tabele krzyżowe pozwalają liczyć zagregowane statystyki dla określonych kombinacji wartości. Przykładowo, rozważmy zbudowaną wcześniej tabelę source_destination i kombinacje kraju wylotu i przylotu. Ograniczymy się do 28 państw członkowskich Unii Europejskiej.

In [41]:
eu_countries = ["Austria",
"Belgium",
"Bulgaria",
"Croatia",
"Cyprus",
"Czech Republic",
"Denmark",
"Estonia",
"Finland",
"France",
"Germany",
"Greece",
"Hungary",
"Ireland",
"Italy",
"Latvia",
"Lithuania",
"Luxembourg",
"Malta",
"Netherlands",
"Poland",
"Portugal",
"Romania",
"Slovakia",
"Slovenia",
"Spain",
"Sweden",
"United Kingdom"]

sd = source_destination[source_destination["Source country"].isin(eu_countries) &
                        source_destination["Destination country"].isin(eu_countries)]

sd.pivot_table(values="Airline", index="Source country", columns="Destination country", aggfunc=len)

Destination country,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czech Republic,Denmark,Estonia,Finland,France,...,Malta,Netherlands,Poland,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden,United Kingdom
Source country,,,,,,,,,,,,,,,,,,,,,
Austria,15,2,3,6,3,1,5,NaN,2,11,...,4,5,4,6,8,1,2,40,4,11
Belgium,2,1,4,5,2,4,5,2,1,34,...,2,1,6,18,4,1,3,57,7,17
Bulgaria,3,4,6,NaN,4,2,1,NaN,1,3,...,NaN,2,2,NaN,3,NaN,NaN,8,NaN,10
Croatia,6,5,NaN,22,NaN,2,6,1,3,11,...,NaN,3,3,NaN,NaN,NaN,NaN,5,9,35
Cyprus,3,2,4,NaN,NaN,NaN,NaN,NaN,NaN,2,...,2,3,2,NaN,3,NaN,NaN,NaN,2,50
Czech Republic,1,4,2,1,NaN,4,4,NaN,3,15,...,2,5,3,2,5,4,1,16,5,16
Denmark,5,5,1,6,NaN,4,23,2,4,11,...,2,11,7,5,1,NaN,2,31,9,22
Estonia,NaN,2,NaN,1,NaN,NaN,2,2,3,1,...,NaN,2,1,NaN,NaN,NaN,NaN,1,4,3
Finland,2,1,1,3,NaN,3,4,3,56,6,...,NaN,3,5,2,NaN,NaN,1,16,22,9


## Ćwiczenie

Posłuż się metodą pivot_table, żeby poznać średnie różnice pomiędzy wysokościami lotnisk.

Wskazówka funkcja mean z biblioteki numpy liczby wartość średnią listy.

In [47]:
from numpy import mean

## Pivot

Często można spotkać się ze sposobem przechowywania danych w tabeli nazywanym *long*, stosowanym przeważnie w  relacyjnych bazach danych. W ramach niego wartości różnych atrybutów przechowywane są w tej samej kolumnie, a atrybut identyfikowany jest przy pomocy dodatkowego identyfikatora. Przykładowo:

In [24]:
d = pd.DataFrame({"name": ["John", "John", "John", "Mary", "Mary", "Mary"],
                  "attribute": ["salary", "position", "room number", "salary", "position", "room number"],
                  "value": [1000, "administration specialist", 2, 1700, "engineer", 22]})
d = d[["name", "attribute", "value"]]
d

,name,attribute,value
0,John,salary,1000
1,John,position,administration specialist
2,John,room number,2
3,Mary,salary,1700
4,Mary,position,engineer
5,Mary,room number,22


Pytanie: jakie zalety może mieć taki format?

Do szybkiego przekształcenia danych do formatu, w którym wartości przechowywane są w osobnych kolumnach, służy metoda pivot:

In [25]:
d1 = d.pivot("name", "attribute", "value")
d1

attribute,position,room number,salary
name,,,
John,administration specialist,2,1000
Mary,engineer,22,1700


Odwrotność pivot stanowi funkcja melt:

In [34]:
d1["name"] = d1.index
pd.melt(d1, id_vars=["name"], value_vars=["position", "room number", "salary"])

,name,attribute,value
0,John,position,administration specialist
1,Mary,position,engineer
2,John,room number,2
3,Mary,room number,22
4,John,salary,1000
5,Mary,salary,1700
